In [5]:
import random
import math
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import Grid
from mesa.datacollection import DataCollector

In [7]:
class TreeCell(Agent):
    '''
    A tree cell.
    
    Attributes:
        x, y: Grid coordinates
        condition: Can be "Fine", "On Fire", or "Burned Out"
        unique_id: (x,y) tuple. 

    '''
    def __init__(self, model, pos):
        '''
        Create a new tree and use tree's coordinates on the grid as the unique_id.
        Initial status is Fine
        '''
        super().__init__(pos, model)
        self.pos = pos
        self.unique_id = pos
        self.condition = "Fine"
        
    def step(self):
        '''
        If the tree is on fire, spread it to fine trees nearby.
        '''
        if self.condition == "On Fire":
            neighbors = self.model.grid.get_neighbors(self.pos, moore=True)
            for neighbor in neighbors:
                if neighbor.condition == "Fine":
                    neighbor.condition = "On Fire"
            self.condition = "Burned Out"
            


In [10]:
class Walker(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.unique_id = unique_id
        self.pos = pos

    def random_move(self):
        '''
        This method should get the neighbouring cells (Moore's neighbourhood)
        select one, and move the agent to this cell.
        '''
        # get all neighbours within reachable distance limited by speed
        cell_list = self.model.grid.get_neighborhood(self.pos, moore=True, radius=self.truck_max_speed)

        # choose the new position by random walk
        new_pos = cell_list[random.randint(0, len(cell_list) - 1)]

        self.model.grid.move_agent(self, new_pos)
        
    def take_step(self, given_neighbor):
        '''This function takes a step in the direction of a given neighbour'''

        # calculates total places to move in x and y direction
        places_to_move_x = given_neighbor.pos[0] - self.pos[0]
        places_to_move_y = given_neighbor.pos[1] - self.pos[1]
        
        # lowers the max speed of the trucks when destination is closer
        speed_x = min(self.truck_max_speed, abs(places_to_move_x))
        speed_y = min(self.truck_max_speed, abs(places_to_move_y))

        new_x, new_y = self.pos[0], self.pos[1]

        # determine new position of agent
        if places_to_move_x > 0:
            new_x += speed_x
        if places_to_move_x < 0:
            new_x -= speed_x
        if places_to_move_y > 0:
            new_y += speed_y
        if places_to_move_y < 0:
            new_y -= speed_y

        self.model.grid.move_agent(self, (new_x, new_y))
        
    def closest_fire_move(self):
        '''Makes agents move towards the closest fire'''
        
        # calculate fire fighter to burning vegetation ratio
        ratio = self.firefighters_tree_ratio(self.model.num_firetruck, 
                                             self.model.count_type(self.model, "On Fire"))
        fire_intheneighborhood = False
        
        # skip through a percentage of the vision to find the closest fire more efficiently
        limited_vision_list = [i for i in range(2, 100, 2)]
        for i in range(len(limited_vision_list)):
            limited_vision = int(self.vision * limited_vision_list[i] / 100.)
        
            if i > 0:
                inner_radius = int(
                    self.vision * limited_vision_list[i - 1] / 100.)
            else:
                inner_radius = 0

            # find hot trees in neighborhood
            neighbors_list = self.model.grid.get_neighbors(
                self.pos, moore=True, radius=limited_vision, inner_radius=inner_radius)

            # filter for trees that are on fire
            neighbors_list = [
                x for x in neighbors_list if x.condition == "On Fire"]

            # find nearest fire
            min_distance = limited_vision ** 2
            for neighbor in neighbors_list:
                if neighbor.trees_claimed < ratio:
                    distance = abs(neighbor.pos[0] ** 2 - self.pos[0] ** 2) + \
                        abs(neighbor.pos[1] ** 2 - self.pos[1] ** 2)
                    if distance < min_distance:
                        min_distance = distance
                        closest_neighbor = neighbor
                        fire_intheneighborhood = True
            if fire_intheneighborhood:
                break

        # move toward fire if it is actually in the neighborhood
        if fire_intheneighborhood:
            self.take_step(closest_neighbor)
            closest_neighbor.trees_claimed += 1

        # if fire not in the neighboorhood, do random move
        else:
            self.random_move()

In [11]:
class Fire(Model):
    '''
    Simple Fire model.
    '''
    def __init__(self, height, width, density):
        '''
        Create a new fire model.
        
        Args:
            height, width: The size of the grid to model
            density: What fraction of grid cells have a tree in them.
        '''
        # Initialize model parameters
        self.height = height
        self.width = width
        self.density = density
        
        # Set up model objects
        self.schedule = RandomActivation(self)
        self.grid = Grid(height, width, torus=False)
        self.dc = DataCollector({"Fine": lambda m: self.count_type(m, "Fine"),
                                "On Fire": lambda m: self.count_type(m, "On Fire"),
                                "Burned Out": lambda m: self.count_type(m, "Burned Out")})
        
        # Place a tree in each cell with Prob = density
        for x in range(self.width):
            for y in range(self.height):
                if random.random() < self.density:
                    # Create a tree
                    new_tree = TreeCell(self, (x, y))
                    # Set all trees in the middle on fire.
                    if x == self.width/2 and y < self.height/2:
                        new_tree.condition = "On Fire"
                    self.grid[y][x] = new_tree
                    self.schedule.add(new_tree)
        self.running = True
        
    def step(self):
        '''
        Advance the model by one step.
        '''
        self.schedule.step()
        self.dc.collect(self)
        # Halt if no more fire
        if self.count_type(self, "On Fire") == 0:
            self.running = False
    
    @staticmethod
    def count_type(model, tree_condition):
        '''
        Helper method to count trees in a given condition in a given model.
        '''
        count = 0
        for tree in model.schedule.agents:
            if tree.condition == tree_condition:
                count += 1
        return count

    def init_firefighters(self, agent_type, num_firetruck,
                          truck_strategy, vision, truck_max_speed, placed_on_edges):
        '''
        Initialises the fire fighters
        placed_on_edges: if True --> places the firetrucks randomly over the grid.
        If False it places the firetrucks equispaced on the rim of the grid.
        '''

        if num_firetruck > 0:

            # Places the firetrucks on the edge of the grid with equal spacing
            if placed_on_edges:
                init_positions = self.equal_spread()
                for i in range(num_firetruck):
                    my_pos = init_positions.pop()
                    firetruck = self.new_firetruck(
                        Firetruck, my_pos, truck_strategy, vision, truck_max_speed)
                    self.schedule_FireTruck.add(firetruck)
                    self.schedule.add(firetruck)
                    self.firefighters_lists.append(firetruck)

            # Places the firetrucks randomly on the grid
            else:
                for i in range(num_firetruck):
                    x = random.randrange(self.width)
                    y = random.randrange(self.height)

                    # make sure fire fighting agents are not placed in a river
                    while self.grid.get_cell_list_contents((x, y)):
                        if isinstance(self.grid.get_cell_list_contents(
                                (x, y))[0], RiverCell):
                            x = random.randrange(self.width)
                            y = random.randrange(self.height)
                        else:
                            break

                    firetruck = self.new_firetruck(
                        Firetruck, (x, y), truck_strategy, vision, truck_max_speed)
                    self.schedule_FireTruck.add(firetruck)
                    self.schedule.add(firetruck)
                    self.firefighters_lists.append(firetruck)
                    
                    
    def step(self):
        '''
        Advance the model by one step.
        '''

        # progress the fire spread by a step
        self.schedule_TreeCell.step()

        # save all burning trees in tree_list
        self.tree_list = [tree for tree in model.schedule_TreeCell.agents if tree.condition == "On Fire"]

        # if using optimized method, produce a matrix with the distances between the firetrucks and the burning veg
        if len(self.tree_list) > 0:
            if (self.truck_strategy == "Optimized closest"):
                self.assigned_list = self.assign_closest(
                    self.compute_distances(self.tree_list,
                                           self.firefighters_lists), self.tree_list)

            elif (self.truck_strategy == "Optimized Parallel attack"):
                self.assigned_list = self.assign_parallel(
                    self.compute_distances(self.tree_list, self.firefighters_lists),
                    self.tree_list)

            elif (self.truck_strategy == "Indirect attack"):
                self.assigned_list = self.assign_parallel(
                    self.compute_distances(self.tree_list, self.firefighters_lists),
                    self.tree_list)

            # progress the firetrucks by one step
            self.schedule_FireTruck.step()

        # collect data
        self.dc.collect(self, [TreeCell, Firetruck]) # because of modified dc, now the agents need to be specified
        self.current_step += 1

        # if spontaneous fires are turned on, check whether one ignites in this step
        if self.random_fires:
            randtree = int(random.random() * len(self.agents))
            if self.agents[randtree].condition == "Fine":
                self.randomfire(self, randtree)

        # Halt if no more fire
        if self.count_type(self, "On Fire") == 0:
            print(" \n \n Fire is gone ! \n \n")
            self.running = False
                    
    def new_firetruck(self, agent_type, pos, truck_strategy,
                      vision, truck_max_speed):
        '''
        Method to add a fire agent.
        '''
        self.n_agents += 1

        # Create a new agent of the given type
        new_agent = agent_type(
            self,
            self.n_agents,
            pos,
            truck_strategy,
            vision,
            truck_max_speed)

        # Place the agent on the grid
        self.grid.place_agent(new_agent, pos)

        # And add the agent to the model so we can track it
        self.agents.append(new_agent)

        return new_agent

    def remove_agent(self, agent):
        '''
        Method that enables us to remove passed agents.
        '''
        self.n_agents -= 1

        # Remove agent from grid
        self.grid.remove_agent(agent)

        # Remove agent from model
        self.agents.remove(agent)
                    
    def equal_spread(self):
        '''
        Function to equally space the firetruck along the edge of the grid
        '''
        edge_len = self.height - 1
        total_edge = 4 * edge_len

        x = 0
        y = 0

        start_pos = [(x, y)]
        spacing = total_edge / self.num_firetruck
        total_edge -= spacing
        step = 0

        while total_edge > 0:
            fill_x = edge_len - x
            fill_y = edge_len - y

            # special cases (<4)
            if spacing > edge_len:
                if x == 0:
                    x += edge_len
                    y += spacing - edge_len
                else:
                    x, y = y, x

            # all other cases
            else:
                # Increasing x
                if y == 0 and x + spacing <= edge_len and step < 2:
                    x += spacing
                    step = 1

                # x maxxed, increasing y
                elif x + spacing > edge_len and y + (spacing - fill_x) < edge_len and step < 3:
                    x += fill_x
                    y += spacing - fill_x
                    step = 2

                # x&y maxxed, decreasing x
                elif x - (spacing - fill_y) >= 0 and y + fill_y >= edge_len and step < 4:
                    x -= (spacing - fill_y)
                    y += fill_y
                    step = 3

                # x emptied, decreasing y
                elif x - spacing < 0 and step < 5:
                    y -= (spacing - x)
                    x = 0
                    step = 4

            start_pos += [(round(x), round(y))]
            total_edge -= spacing

        return start_pos

In [12]:
fire = Fire(100, 100, 0.8)
fire.run_model()